In [1]:
#### import libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# date and time utilities from the standard library
from datetime import datetime, date, timedelta
from sklearn.metrics import mean_absolute_error
from lightgbm import LGBMRegressor
# library to suppress warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

# custom library 
# import enefit

In [2]:
#### update data directory path
DATA_DIR = '/kaggle/input/predict-energy-behavior-of-prosumers'

In [3]:
#### get the full path of all files
# walk through the every directory inside DATA_DIR 
for directory, subdirectories, files in os.walk(DATA_DIR):
    # within each directory, iterate over every filename
    for file in files:
        # join the directory path and filename
        # by using os.path.join(), you make your code portable across different operating systems
        print(os.path.join(directory, file))

/kaggle/input/predict-energy-behavior-of-prosumers/client.csv
/kaggle/input/predict-energy-behavior-of-prosumers/gas_prices.csv
/kaggle/input/predict-energy-behavior-of-prosumers/electricity_prices.csv
/kaggle/input/predict-energy-behavior-of-prosumers/weather_station_to_county_mapping.csv
/kaggle/input/predict-energy-behavior-of-prosumers/public_timeseries_testing_util.py
/kaggle/input/predict-energy-behavior-of-prosumers/historical_weather.csv
/kaggle/input/predict-energy-behavior-of-prosumers/county_id_to_name_map.json
/kaggle/input/predict-energy-behavior-of-prosumers/train.csv
/kaggle/input/predict-energy-behavior-of-prosumers/forecast_weather.csv
/kaggle/input/predict-energy-behavior-of-prosumers/example_test_files/sample_submission.csv
/kaggle/input/predict-energy-behavior-of-prosumers/example_test_files/client.csv
/kaggle/input/predict-energy-behavior-of-prosumers/example_test_files/gas_prices.csv
/kaggle/input/predict-energy-behavior-of-prosumers/example_test_files/electricity

In [4]:
#### read the CSV files into DataFrames
train = pd.read_csv(os.path.join(DATA_DIR, "train.csv"))
gas_df = pd.read_csv(os.path.join(DATA_DIR, "gas_prices.csv"))
electricity_df = pd.read_csv(os.path.join(DATA_DIR, "electricity_prices.csv"))
client_df = pd.read_csv(os.path.join(DATA_DIR, "client.csv"))
fw_df = pd.read_csv(os.path.join(DATA_DIR, "forecast_weather.csv"))
hw_df = pd.read_csv(os.path.join(DATA_DIR, "historical_weather.csv"))

# read a file from a different directory
# see how this data is generated: https://www.kaggle.com/jackren000/mapping-locations-and-county-code/edit
locations = pd.read_csv("/kaggle/input/county-lon-lats/county_lon_lats.csv")

#### 1. train.csv

In [5]:
train.head()

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id
0,0,0,1,0.713,0,2021-09-01 00:00:00,0,0,0
1,0,0,1,96.590,1,2021-09-01 00:00:00,0,1,0
2,0,0,2,0.000,0,2021-09-01 00:00:00,0,2,1
3,0,0,2,17.314,1,2021-09-01 00:00:00,0,3,1
4,0,0,3,2.904,0,2021-09-01 00:00:00,0,4,2


In [6]:
print(f'The shape of the dataset is: {train.shape}')
print(f'\nThe info of the dateset is:')
print(train.info())

The shape of the dataset is: (2018352, 9)

The info of the dateset is:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2018352 entries, 0 to 2018351
Data columns (total 9 columns):
 #   Column              Dtype  
---  ------              -----  
 0   county              int64  
 1   is_business         int64  
 2   product_type        int64  
 3   target              float64
 4   is_consumption      int64  
 5   datetime            object 
 6   data_block_id       int64  
 7   row_id              int64  
 8   prediction_unit_id  int64  
dtypes: float64(1), int64(7), object(1)
memory usage: 138.6+ MB
None


In [7]:
gas_df.head()

,forecast_date,lowest_price_per_mwh,highest_price_per_mwh,origin_date,data_block_id
0,2021-09-01,45.23,46.32,2021-08-31,1
1,2021-09-02,45.62,46.29,2021-09-01,2
2,2021-09-03,45.85,46.40,2021-09-02,3
3,2021-09-04,46.30,46.80,2021-09-03,4
4,2021-09-05,46.30,46.58,2021-09-04,5


In [8]:
hw_df.head()

,datetime,temperature,dewpoint,rain,snowfall,surface_pressure,cloudcover_total,cloudcover_low,cloudcover_mid,cloudcover_high,windspeed_10m,winddirection_10m,shortwave_radiation,direct_solar_radiation,diffuse_radiation,latitude,longitude,data_block_id
0,2021-09-01 00:00:00,14.2,11.6,0.0,0.0,1015.9,31,31,0,11,7.083333,8,0.0,0.0,0.0,57.6,21.7,1.0
1,2021-09-01 00:00:00,13.9,11.5,0.0,0.0,1010.7,33,37,0,0,5.111111,359,0.0,0.0,0.0,57.6,22.2,1.0
2,2021-09-01 00:00:00,14.0,12.5,0.0,0.0,1015.0,31,34,0,0,6.333333,355,0.0,0.0,0.0,57.6,22.7,1.0
3,2021-09-01 00:00:00,14.6,11.5,0.0,0.0,1017.3,0,0,0,0,8.083333,297,358.0,277.0,81.0,57.6,23.2,1.0
4,2021-09-01 00:00:00,15.7,12.9,0.0,0.0,1014.0,22,25,0,0,8.416667,5,0.0,0.0,0.0,57.6,23.7,1.0


Preprocessing Data

#### electricity_prices.csv

In [9]:
electricity_df.head()

,forecast_date,euros_per_mwh,origin_date,data_block_id
0,2021-09-01 00:00:00,92.51,2021-08-31 00:00:00,1
1,2021-09-01 01:00:00,88.90,2021-08-31 01:00:00,1
2,2021-09-01 02:00:00,87.35,2021-08-31 02:00:00,1
3,2021-09-01 03:00:00,86.88,2021-08-31 03:00:00,1
4,2021-09-01 04:00:00,88.43,2021-08-31 04:00:00,1


In [10]:
################## electricity
# rename 'forecast_date' column to 'datetime' for consistency before merging
electricity_df = electricity_df.rename(columns={'forecast_date': 'datetime'}) 

# convert datetime to UTC
electricity_df['datetime'] = pd.to_datetime(electricity_df['datetime'], utc=True)

# add hour column
electricity_df['hour'] = electricity_df['datetime'].dt.hour
electricity_df.head()

,datetime,euros_per_mwh,origin_date,data_block_id,hour
0,2021-09-01 00:00:00+00:00,92.51,2021-08-31 00:00:00,1,0
1,2021-09-01 01:00:00+00:00,88.90,2021-08-31 01:00:00,1,1
2,2021-09-01 02:00:00+00:00,87.35,2021-08-31 02:00:00,1,2
3,2021-09-01 03:00:00+00:00,86.88,2021-08-31 03:00:00,1,3
4,2021-09-01 04:00:00+00:00,88.43,2021-08-31 04:00:00,1,4


#### forecast_weather.csv

In [11]:
fw_df.head(100)

,latitude,longitude,origin_datetime,hours_ahead,temperature,dewpoint,cloudcover_high,cloudcover_low,cloudcover_mid,cloudcover_total,10_metre_u_wind_component,10_metre_v_wind_component,data_block_id,forecast_datetime,direct_solar_radiation,surface_solar_radiation_downwards,snowfall,total_precipitation
0,57.6,21.7,2021-09-01 02:00:00,1,15.655786,11.553613,0.904816,0.019714,0.000000,0.905899,-0.411328,-9.106137,1,2021-09-01 03:00:00,0.0,0.0,0.0,0.0
1,57.6,22.2,2021-09-01 02:00:00,1,13.003931,10.689844,0.886322,0.004456,0.000000,0.886658,0.206347,-5.355405,1,2021-09-01 03:00:00,0.0,0.0,0.0,0.0
2,57.6,22.7,2021-09-01 02:00:00,1,14.206567,11.671777,0.729034,0.005615,0.000000,0.730499,1.451587,-7.417905,1,2021-09-01 03:00:00,0.0,0.0,0.0,0.0
3,57.6,23.2,2021-09-01 02:00:00,1,14.844507,12.264917,0.336304,0.074341,0.000626,0.385468,1.090869,-9.163999,1,2021-09-01 03:00:00,0.0,0.0,0.0,0.0
4,57.6,23.7,2021-09-01 02:00:00,1,15.293848,12.458887,0.102875,0.088074,0.000015,0.176590,1.268481,-8.975766,1,2021-09-01 03:00:00,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,59.4,27.2,2021-09-01 02:00:00,1,13.360986,11.038843,0.194427,0.330963,0.322891,0.514740,1.813769,-4.185727,1,2021-09-01 03:00:00,0.0,0.0,0.0,0.0
96,59.4,27.7,2021-09-01 02:00:00,1,14.429102,12.208032,0.097198,0.210419,0.278946,0.400513,1.723315,-5.393003,1,2021-09-01 03:00:00,0.0,0.0,0.0,0.0
97,59.4,28.2,2021-09-01 02:00:00,1,13.080103,12.063135,0.019257,0.165039,0.232071,0.280624,2.276172,-1.788999,1,2021-09-01 03:00:00,0.0,0.0,0.0,0.0
98,59.7,21.7,2021-09-01 02:00:00,1,14.834619,11.027856,0.677979,0.000366,0.000000,0.678116,2.989673,-9.116879,1,2021-09-01 03:00:00,0.0,0.0,0.0,0.0


In [12]:
locations.head(100)

,Unnamed: 0,county,longitude,latitude
0,0,0,24.2,59.1
1,1,0,25.2,59.1
2,2,0,23.7,59.4
3,3,0,24.2,59.4
4,4,0,24.7,59.4
...,...,...,...,...
70,70,15,26.7,57.6
71,71,15,27.2,57.6
72,72,15,26.7,57.9
73,73,15,27.2,57.9


In [13]:
################## forecast_weather
# round 'latitude' and 'longitude' to 1 decimal place
fw_df[['latitude', 'longitude']] = fw_df[['latitude', 'longitude']].astype(float).round(1)

# merge 'forecast_weather' with 'locations' on coordinates to add county information
fw_df = fw_df.merge(locations, how='left', on=['longitude', 'latitude'])

# drop missing values
fw_df.dropna(axis=0, inplace=True)

# convert the 'county' column to integer data type
fw_df['county'] = fw_df['county'].astype('int64')

# drop the specified columns as they are not needed for further analysis
# latitude and longitude are used for mapping the county codes
fw_df.drop(['origin_datetime', 'latitude', 'longitude', 'hours_ahead', 'data_block_id'], axis=1, inplace=True)

# rename the 'forecast_datetime' column to 'datetime' for consistency with other datasets
fw_df.rename(columns={'forecast_datetime': 'datetime'}, inplace=True)

# convert the 'datetime' column to UTC
fw_df['datetime'] = pd.to_datetime(fw_df['datetime'], utc=True)

In [14]:
fw_df.head(100)

,temperature,dewpoint,cloudcover_high,cloudcover_low,cloudcover_mid,cloudcover_total,10_metre_u_wind_component,10_metre_v_wind_component,datetime,direct_solar_radiation,surface_solar_radiation_downwards,snowfall,total_precipitation,Unnamed: 0,county
3,14.844507,12.264917,0.336304,0.074341,0.000626,0.385468,1.090869,-9.163999,2021-09-01 03:00:00+00:00,0.0,0.0,0.0,0.000000e+00,50.0,10
10,9.876489,9.120386,0.000000,0.025940,0.003006,0.026001,1.048755,-2.526303,2021-09-01 03:00:00+00:00,0.0,0.0,0.0,0.000000e+00,70.0,15
11,10.815576,10.046899,0.747406,0.083191,0.000000,0.761780,1.183276,-3.873715,2021-09-01 03:00:00+00:00,0.0,0.0,0.0,0.000000e+00,71.0,15
14,16.050439,12.355493,0.886078,0.051636,0.000092,0.889587,0.426929,-10.098569,2021-09-01 03:00:00+00:00,0.0,0.0,0.0,0.000000e+00,51.0,10
15,14.418481,11.374536,0.855011,0.003052,0.000015,0.855301,0.902514,-6.924008,2021-09-01 03:00:00+00:00,0.0,0.0,0.0,0.000000e+00,52.0,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,11.062158,9.959985,0.198441,0.999542,0.412964,0.999664,1.941915,-3.570616,2021-09-01 04:00:00+00:00,0.0,0.0,0.0,7.450581e-07,67.0,14
149,11.487207,10.202783,0.252960,0.558014,0.314087,0.684052,1.950948,-2.862852,2021-09-01 04:00:00+00:00,0.0,0.0,0.0,1.490116e-07,62.0,11
150,10.042505,8.659204,0.137680,0.452209,0.140244,0.547058,3.034567,-2.149473,2021-09-01 04:00:00+00:00,0.0,0.0,0.0,0.000000e+00,63.0,11
151,9.948999,8.287012,0.001892,0.215851,0.268799,0.307404,2.855245,-1.884825,2021-09-01 04:00:00+00:00,0.0,0.0,0.0,0.000000e+00,46.0,8


In [15]:
################## forecast_weather_datetime
# 1. convert the 'datetime' column to a period indexd by hour and assign it to a new variable
hour_period = fw_df['datetime'].dt.to_period('h')

# 2. drop the columns we don't want to include in the mean calculation
columns_to_keep = fw_df.drop(['county', 'datetime'], axis=1).columns

# 3. group by the hourly period and calculate the mean for the remaining columns
grouped = fw_df.groupby(hour_period)[columns_to_keep].mean()

# 4. reset the index so we can have the hourly periods as a regular column (add an index column)
fw_df_datetime = grouped.reset_index()

# convert hourly periods back to timestamps
# fw_df_datetime contains the average values for each hour, across all dates
fw_df_datetime['datetime'] = pd.to_datetime(fw_df_datetime['datetime'].dt.to_timestamp(), utc=True)

In [16]:
fw_df_datetime.head()

,datetime,temperature,dewpoint,cloudcover_high,cloudcover_low,cloudcover_mid,cloudcover_total,10_metre_u_wind_component,10_metre_v_wind_component,direct_solar_radiation,surface_solar_radiation_downwards,snowfall,total_precipitation,Unnamed: 0
0,2021-09-01 03:00:00+00:00,13.014066,10.531545,0.256954,0.213331,0.069662,0.438023,1.970332,-5.205125,0.000000,0.000000,0.0,0.000000e+00,37.0
1,2021-09-01 04:00:00+00:00,12.891813,10.474813,0.278619,0.178085,0.079571,0.427232,2.092311,-5.197120,0.000000,0.000000,0.0,1.151164e-06,37.0
2,2021-09-01 05:00:00+00:00,12.693667,10.327287,0.244363,0.120304,0.068174,0.368063,2.319048,-5.227491,0.000000,0.000000,0.0,8.145968e-07,37.0
3,2021-09-01 06:00:00+00:00,12.666698,10.257059,0.244026,0.108244,0.088197,0.380928,2.460921,-5.257574,57.310637,9.754364,0.0,7.553895e-07,37.0
4,2021-09-01 07:00:00+00:00,13.212046,10.436997,0.216526,0.117397,0.073299,0.344576,2.581852,-5.341152,257.277830,82.380557,0.0,1.347065e-06,37.0


In [17]:
################## forecast_weather_datetime_county
# 1. convert the 'datetime' column to a period indexd by hour and assign it to a new variable
hour_period = fw_df['datetime'].dt.to_period('h')

# 2. drop the columns we don't want to include in the mean calculation
columns_to_keep = fw_df.drop(['county', 'datetime'], axis=1).columns

# 3. group by the hourly period and calculate the mean for the remaining columns
grouped = fw_df.groupby([hour_period, 'county'])[columns_to_keep].mean()

# 4. reset the index so we can have the hourly periods as a regular column (add an index column)
fw_df_datetime_county = grouped.reset_index()

# convert hourly periods back to timestamps
# fw_df_datetime contains the average values for each county per hour, across all dates
fw_df_datetime_county['datetime'] = pd.to_datetime(fw_df_datetime_county['datetime'].dt.to_timestamp(), utc=True)

#### historical_weather.csv

In [18]:
hw_df.head()

,datetime,temperature,dewpoint,rain,snowfall,surface_pressure,cloudcover_total,cloudcover_low,cloudcover_mid,cloudcover_high,windspeed_10m,winddirection_10m,shortwave_radiation,direct_solar_radiation,diffuse_radiation,latitude,longitude,data_block_id
0,2021-09-01 00:00:00,14.2,11.6,0.0,0.0,1015.9,31,31,0,11,7.083333,8,0.0,0.0,0.0,57.6,21.7,1.0
1,2021-09-01 00:00:00,13.9,11.5,0.0,0.0,1010.7,33,37,0,0,5.111111,359,0.0,0.0,0.0,57.6,22.2,1.0
2,2021-09-01 00:00:00,14.0,12.5,0.0,0.0,1015.0,31,34,0,0,6.333333,355,0.0,0.0,0.0,57.6,22.7,1.0
3,2021-09-01 00:00:00,14.6,11.5,0.0,0.0,1017.3,0,0,0,0,8.083333,297,358.0,277.0,81.0,57.6,23.2,1.0
4,2021-09-01 00:00:00,15.7,12.9,0.0,0.0,1014.0,22,25,0,0,8.416667,5,0.0,0.0,0.0,57.6,23.7,1.0


In [19]:
################## hist_weather
# round 'latitude' and 'longitude' to 1 decimal place
hw_df[['latitude', 'longitude']] = hw_df[['latitude', 'longitude']].astype(float).round(1)

# add county information to 'hist_weather' based on matching coordinates
hw_df = hw_df.merge(locations, how='left', on=['longitude', 'latitude'])

# remove rows with any missing values
hw_df.dropna(axis=0, inplace=True)

# remove 'latitude' and 'longitude' columns as they are no longer needed
hw_df.drop(['latitude', 'longitude'], axis=1, inplace=True)

# change the 'county' column to integer datatype
hw_df['county'] = hw_df['county'].astype('int64')

# convert 'datetime' column to UTC
hw_df['datetime'] = pd.to_datetime(hw_df['datetime'], utc=True)

In [20]:
hw_df.head()

,datetime,temperature,dewpoint,rain,snowfall,surface_pressure,cloudcover_total,cloudcover_low,cloudcover_mid,cloudcover_high,windspeed_10m,winddirection_10m,shortwave_radiation,direct_solar_radiation,diffuse_radiation,data_block_id,Unnamed: 0,county
3,2021-09-01 00:00:00+00:00,14.6,11.5,0.0,0.0,1017.3,0,0,0,0,8.083333,297,358.0,277.0,81.0,1.0,50.0,10
10,2021-09-01 00:00:00+00:00,13.0,11.5,0.0,0.0,998.5,100,85,17,98,4.027778,354,0.0,0.0,0.0,1.0,70.0,15
11,2021-09-01 00:00:00+00:00,13.0,12.1,0.1,0.0,988.9,100,100,39,99,3.805556,360,0.0,0.0,0.0,1.0,71.0,15
14,2021-09-01 00:00:00+00:00,13.3,10.7,0.0,0.0,1016.2,51,35,0,64,7.611111,5,0.0,0.0,0.0,1.0,51.0,10
15,2021-09-01 00:00:00+00:00,14.4,11.3,0.0,0.0,1016.2,51,35,0,64,7.611111,5,0.0,0.0,0.0,1.0,52.0,10


In [21]:
################## hist_weather_datetime
# 1. convert the 'datetime' column to a period indexd by hour and assign it to a new variable
hour_period = hw_df['datetime'].dt.to_period('h')

# 2. drop the columns we don't want to include in the mean calculation
columns_to_keep = hw_df.drop(['county', 'datetime', 'data_block_id'], axis=1).columns

# 3. group by the hourly period and calculate the mean for the remaining columns
grouped = hw_df.groupby(hour_period)[columns_to_keep].mean()

# 4. reset the index so we can have the hourly periods as a regular column
hw_df_datetime = grouped.reset_index()

# convert hourly periods back to timestamps
# hw_df_datetime contains the average values for each hour, across all dates
hw_df_datetime['datetime'] = pd.to_datetime(hw_df_datetime['datetime'].dt.to_timestamp(), utc=True)
hw_df_datetime = hw_df_datetime.merge(hw_df[['datetime', 'data_block_id']], how='left', on='datetime')

In [22]:
################## hist_weather_datetime_county
# 1. convert the 'datetime' column to a period indexd by hour and assign it to a new variable
hour_period = hw_df['datetime'].dt.to_period('h')

# 2. drop the columns we don't want to include in the mean calculation
columns_to_keep = hw_df.drop(['county', 'datetime', 'data_block_id'], axis=1).columns

# 3. group by the hourly period and calculate the mean for the remaining columns
grouped = hw_df.groupby(['county', hour_period])[columns_to_keep].mean()

# 4. reset the index so we can have the hourly periods as a regular column
hw_df_datetime_county = grouped.reset_index()

# convert hourly periods back to timestamps
# hw_df_datetime contains the average values for each hour, across all dates
hw_df_datetime_county['datetime'] = pd.to_datetime(hw_df_datetime_county['datetime'].dt.to_timestamp(), utc=True)
hw_df_datetime_county = hw_df_datetime_county.merge(hw_df[['datetime', 'data_block_id']], how='left', on='datetime')

In [23]:
# extract the hour from the 'datetime' column and create a new 'hour' column in hw_df_datetime
hw_df_datetime['hour'] = hw_df_datetime['datetime'].dt.hour
# extract the hour from the 'datetime' column and create a new 'hour' column
hw_df_datetime_county['hour'] = hw_df_datetime_county['datetime'].dt.hour
# remove duplicate rows, if any, based on all columns
hw_df_datetime.drop_duplicates(inplace=True)
# remove duplicate rows, if any, based on all columns
hw_df_datetime_county.drop_duplicates(inplace=True)
# drop the 'datetime' column as it's no longer needed after extracting hour
hw_df_datetime.drop('datetime', axis=1, inplace=True)
# drop the 'datetime' column as it's no longer needed after extracting hour
hw_df_datetime_county.drop('datetime', axis=1, inplace=True)

In [24]:
'''################## data
# drop missing values of target 
data.dropna(subset=['target'], inplace=True)

# convert datetime to UTC
data['datetime'] = pd.to_datetime(data['datetime'], utc=True)

# add year column in train data
data['year'] = data['datetime'].dt.year

# add month column
data['month'] = data['datetime'].dt.month

# add hour column
data['hour'] = data['datetime'].dt.hour

# add day of week column
data['dayofweek'] = data['datetime'].dt.dayofweek

# add day of year column
data['dayofyear'] = data['datetime'].dt.dayofyear


################## electricity
# rename 'forecast_date' column to 'datetime' for consistency before merging
electricity = electricity.rename(columns={'forecast_date': 'datetime'}) 

# convert datetime to UTC
electricity['datetime'] = pd.to_datetime(electricity['datetime'], utc=True)

# add hour column
electricity['hour'] = electricity['datetime'].dt.hour


################## client
# reduce 'block_id' in the 'client' DataFrame by 2 to match 'train' DataFrame
client['data_block_id'] -= 2


################## location
# remove the 'Unnamed: 0' column from the 'locations' DataFrame
locations.drop('Unnamed: 0', axis=1, inplace=True)


################## forecast_weather
# round 'latitude' and 'longitude' to 1 decimal place
forecast_weather[['latitude', 'longitude']] = forecast_weather[['latitude', 'longitude']].astype(float).round(1)

# merge 'forecast_weather' with 'locations' on coordinates to add county information
forecast_weather = forecast_weather.merge(locations, how='left', on=['longitude', 'latitude'])

# drop missing values
forecast_weather.dropna(axis=0, inplace=True)

# convert the 'county' column to integer data type
forecast_weather['county'] = forecast_weather['county'].astype('int64')

# drop the specified columns as they are not needed for further analysis
forecast_weather.drop(['origin_datetime', 'latitude', 'longitude', 'hours_ahead', 'data_block_id'], axis=1, inplace=True)

# rename the 'forecast_datetime' column to 'datetime' for consistency with other datasets
forecast_weather.rename(columns={'forecast_datetime': 'datetime'}, inplace=True)

# convert the 'datetime' column to UTC
forecast_weather['datetime'] = pd.to_datetime(forecast_weather['datetime'], utc=True)

################## forecast_weather_datetime
# 1. convert the 'datetime' column to a period indexd by hour and assign it to a new variable
hour_period = forecast_weather['datetime'].dt.to_period('h')

# 2. drop the columns we don't want to include in the mean calculation
columns_to_keep = forecast_weather.drop(['county', 'datetime'], axis=1).columns

# 3. group by the hourly period and calculate the mean for the remaining columns
grouped = forecast_weather.groupby(hour_period)[columns_to_keep].mean()

# 4. reset the index so we can have the hourly periods as a regular column
forecast_weather_datetime = grouped.reset_index()

forecast_weather_datetime['datetime'] = pd.to_datetime(forecast_weather_datetime['datetime'].dt.to_timestep(), utc=True)

forecast_weather_datetime_county = forecast_weather.groupby(['county', forecast_weather['datetime'].dt.to_period('h')])

################## hist_weather
# round 'latitude' and 'longitude' to 1 decimal place
hist_weather[['latitude', 'longitude']] = hist_weather[['latitude', 'longitude']].astype(float).round(1)

# add county information to 'hist_weather' based on matching coordinates
hist_weather = hist_weather.merge(locations, how='left', on=['longitude', 'latitude'])

# remove rows with any missing values
hist_weather.dropna(axis=0, inplace=True)

# remove 'latitude' and 'longitude' columns as they are no longer needed
hist_weather.drop(['latitude', 'longitude'], axis=1, inplace=True)

# change the 'county' column to integer datatype
hist_weather['county'] = hist_weather['county'].astype('int64')

# convert 'datetime' column to UTC
hist_weather['datetime'] = pd.to_datetime(hist_weather['datetime'], utc=True)

################## hist_weather_datetime
# 1. convert the 'datetime' column to a period indexd by hour and assign it to a new variable
hour_period = hist_weather['datetime'].dt.to_period('h')

# 2. drop the columns we don't want to include in the mean calculation
columns_to_keep = hist_weather.drop(['county', 'datetime', 'data_block_id'], axis=1).columns

# 3. group by the hourly period and calculate the mean for the remaining columns
grouped = hist_weather.groupby(hour_period)[columns_to_keep].mean()

# 4. reset the index so we can have the hourly periods as a regular column
hist_weather_datetime = grouped.reset_index()

# ?????????
hist_weather_datetime['datetime'] = pd.to_datetime(hist_weather_datetime['datetime'].dt.to_timestep(), utc=True)

# 
hist_weather_datetime = hist_weather_datetime.merge(hist_weather[['datetime', 'data_block_id']], how=left, on='datetime')

# add hour column
hist_weather_datetime['hour'] = hist_weather_datetime['datetime'].dt.hour

# add hour column
hist_weather_datetime_county['hour'] = hist_weather_datetime_county['datetime'].dt.hour
                  

# drop duplicated column
hist_weather_datetime.drop_duplicates(inplace=True)

hist_weather_datetime_county.drop_duplicates(inplace=True)

# drop datetime column
hist_weather_datetime.drop('datetime', axis=1, inplace=True)

hist_weather_datetime_county.drop('datetime', axis=1, inplace=True)


################## merge train data with client data, gas data, electricity data, and ect...
data = data.merge(client.drop(columns = ['date']), how='left', on=['data_block_id', 'county', 'is_business', 'product_type'])
data = data.merge(gas[['data_block_id', 'lowest_price_per_mwh', 'highest_pice_per_mwh']], how='left, on='data_block_id')
data = data.merge(electricity[['euros_per_mwh', 'hour', 'data_block_id']], how='left', on=['hour', data_block_id])                  
data = data.merge(forecast_weather_datetime, how='left', on=['datetime'])                  
#????? data = data.merge(forecast_weather_datetime_county, how='left', on=['datetime', 'county'], suffixes=())      
data = data.merge(hist_weather_datetime, how='left', on=['data_block_id', 'hour'])
#????? data = data.merge(hist_weather_datetime_county, how='left', on=['data_block_id', 'county', 'hour'],)                  
                  
#????? data = data.groupby(['year', 'day', 'hour'], as_index=False)  
'''

"################## data\n# drop missing values of target \ndata.dropna(subset=['target'], inplace=True)\n\n# convert datetime to UTC\ndata['datetime'] = pd.to_datetime(data['datetime'], utc=True)\n\n# add year column in train data\ndata['year'] = data['datetime'].dt.year\n\n# add month column\ndata['month'] = data['datetime'].dt.month\n\n# add hour column\ndata['hour'] = data['datetime'].dt.hour\n\n# add day of week column\ndata['dayofweek'] = data['datetime'].dt.dayofweek\n\n# add day of year column\ndata['dayofyear'] = data['datetime'].dt.dayofyear\n\n\n################## electricity\n# rename 'forecast_date' column to 'datetime' for consistency before merging\nelectricity = electricity.rename(columns={'forecast_date': 'datetime'}) \n\n# convert datetime to UTC\nelectricity['datetime'] = pd.to_datetime(electricity['datetime'], utc=True)\n\n# add hour column\nelectricity['hour'] = electricity['datetime'].dt.hour\n\n\n################## client\n# reduce 'block_id' in the 'client' Data

In [25]:
# def preTrain(data, client, histweather)